# PPO for transformer models
> A Pytorch implementation of Proximal Policy Optimization for transfomer models.

This follows the language model approach proposed in paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf) and is similar to the [original implementation](https://github.com/openai/lm-human-preferences). The two main differences are 1) the method is implemented in Pytorch and 2) works with the `transformer` library by Hugging Face.

In [1]:
# default_exp ppo
%load_ext autoreload
%autoreload 2

In [16]:
# export
import numpy as np
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torch
import collections
import time
import random
import transformers

from trl.core import (logprobs_from_logits,
                     whiten,
                     clip_by_value,
                     entropy_from_logits,
                     flatten_dict,
                     average_torch_dicts,
                     stats_to_np,
                     stack_dicts,
                     add_suffix,
                      split_batch_encoding
                     )

from trl.ppo import PPOTrainer
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.pegasus import PegasusWithValueHeadModel, respond_to_batch_pegasus
from transformers import GPT2Tokenizer
from transformers import PegasusTokenizer

In [44]:
gpt_config = {
    "lm_name": "lvwerra/gpt2-imdb",
    "ref_lm_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/bert-imdb",
    "tk_name": "gpt2",
    "steps": 25600,
    "batch_size": 256,
    "forward_batch_size": 16,
    "ppo_epochs": 4,   
    "txt_in_len": 5,
    "txt_out_len": 15,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "device": 'cuda:0'
}

pegasus_config = {
    "lm_name": 'tuner007/pegasus_paraphrase',
    "ref_lm_name": 'tuner007/pegasus_paraphrase',
    "cls_model_name": None,
    "tk_name": "tuner007/pegasus_paraphrase",
    "steps": 25600,
    "batch_size": 2,
    "forward_batch_size": 1,
    "ppo_epochs": 4,   
    "txt_in_len": 5,
    "txt_out_len": 15,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "device_ppo": 'cuda:0',
    "device_ref": 'cuda:1'
}

## KL-controllers
To ensure that the learned policy does not deviate to much from the original language model the KL divergence between the policy and a reference policy (the language model before PPO training) is used as an additional reward signal. Large KL-divergences are punished and staying close to the reference is rewarded.

Two controllers are presented in the paper: an adaptive log-space proportional controller and a fixed controller.

In [26]:
# exports

class AdaptiveKLController:
    """
    Adaptive KL controller described in the paper:
    https://arxiv.org/pdf/1909.08593.pdf
    """
    def __init__(self, init_kl_coef, target, horizon):
        self.value = init_kl_coef
        self.target = target
        self.horizon = horizon

    def update(self, current, n_steps):
        target = self.target
        proportional_error = np.clip(current / target - 1, -0.2, 0.2)
        mult = 1 + proportional_error * n_steps / self.horizon
        self.value *= mult

In [27]:
# exports 

class FixedKLController:
    """Fixed KL controller."""
    def __init__(self, kl_coef):
        self.value = kl_coef

    def update(self, current, n_steps):
        pass

In [ ]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(gpt_config['lm_name']).to(gpt_config['device'])
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(gpt_config['ref_lm_name']).to(gpt_config['device'])
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt_config['tk_name'])

In [28]:
pegasus_model = PegasusWithValueHeadModel.from_pretrained(pegasus_config['lm_name']).to(pegasus_config['device_ppo'])
pegasus_model.train()

pegasus_model_ref = PegasusWithValueHeadModel.from_pretrained(pegasus_config['ref_lm_name']).to(pegasus_config['device_ref'])
pegasus_tokenizer = PegasusTokenizer.from_pretrained(pegasus_config['tk_name'])


Some weights of PegasusWithValueHeadModel were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['v_head.bias', 'v_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
# gpt_ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **gpt_config)
# pegasus_model.to(pegasus_config['device_ppo'])
pegasus_ppo_trainer = PegasusPPOTrainer(pegasus_model, pegasus_model_ref, **pegasus_config)


In [59]:
query_sents = ['there are facilitate your involvement.',
               'A massive glacier had crashed down the mountain mountain.']
resp_sents = ['links that are helpful can be found below.',
              'below are some useful links to facilitate your involvement.' ]

In [60]:
batch_input = pegasus_tokenizer(query_sents ,
                                truncation=True,
                                padding='longest',
                                max_length=60, 
                                return_tensors="pt")

batch_input['decoder_input_ids'] = torch.zeros(batch_input["input_ids"].shape[0], 1, dtype=int)

batch_input = batch_input.to(pegasus_config['device_ppo'])

In [61]:
respond_to_batch_pegasus(pegasus_model, batch_input, txt_len=10)

tensor([[  353,   127,   771,   112,  4721,   119,   107,     1, 95637,  8962],
        [  353,   140,   114,  2926, 23682,  4897,   308,   124,   109,  2924]],
       device='cuda:0')

In [62]:
batch_input

{'input_ids': tensor([[  186,   127,  5186,   128,  5597,   107,     1,     0,     0,     0,
             0],
        [  202,  2926, 23682,   196, 14726,   308,   109,  2924,  2924,   107,
             1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'decoder_input_ids': tensor([[    0,   353,   127,   771,   112,  4721,   119,   107,     1, 95637,
          8962],
        [    0,   353,   140,   114,  2926, 23682,  4897,   308,   124,   109,
          2924]], device='cuda:0')}

In [63]:
pegasus_ppo_trainer.step(batch_input, torch.tensor([3,4]).to(pegasus_config['device_ppo']))


rew:  torch.Size([2, 10]) score:  torch.Size([2])


{'objective/kl': array(-1.0031183, dtype=float32),
 'objective/kl_dist': array([[-0.3044207 , -0.2892434 , -0.05748928,  0.04450554,  0.22243619,
         -0.2094698 ,  0.01841021, -0.01294321,  0.50394726,  0.66732883],
        [-0.11025667, -0.10589361,  0.01775954, -0.51187277,  0.07957881,
         -0.6061425 ,  0.14510864, -1.5177507 ,  0.0041457 ,  0.01602513]],
       dtype=float32),
 'objective/logprobs': array([[ -2.254178  ,  -0.9052554 ,  -0.68954456,  -0.79264885,
          -2.7465656 ,  -2.7636127 ,  -0.1910589 ,  -0.12111196,
         -12.969026  , -11.552953  ],
        [ -2.2795854 ,  -0.38646895,  -0.11493959,  -2.8519258 ,
          -0.10873563,  -6.8220506 ,  -0.53782946,  -6.6941223 ,
          -0.13950907,  -0.05753668]], dtype=float32),
 'objective/ref_logprobs': array([[ -1.9497573 ,  -0.616012  ,  -0.6320553 ,  -0.8371544 ,
          -2.9690018 ,  -2.554143  ,  -0.20946911,  -0.10816875,
         -13.472973  , -12.220282  ],
        [ -2.1693287 ,  -0.28057534, 

In [107]:
split_batchencoding(batch_input, 0,1)

{'input_ids': tensor([[ 487,  127,  181, 1498, 1784,  112, 5186,  128, 5597,  107,    1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'decoder_input_ids': tensor([[    0,  5870,   119, 58128,   137,   258,  1784,   120,   138,  5186,
           128,  5597, 14230, 74594,   296,  2007, 28992,     1,     1,     1,
             1]], device='cuda:0')}

In [65]:
query_tokens_ids = [gpt2_tokenizer.encode(s, return_tensors='pt') for s in query_sents]
query_tokens_ids = torch.cat(query_tokens_ids).to(gpt_config['device'])
query_tokens_ids


tensor([[35993,   389,   617,  4465,  6117,   284, 15570,   534,  9750,    13],
        [   32,  4858, 44539,   550, 14997,   866,   262,  8598,  8598,    13]],
       device='cuda:1')

In [70]:
response_token_ids = respond_to_batch(gpt2_model, query_tokens_ids,txt_len=10)


In [75]:
model_input_gpt = torch.cat((query_tokens_ids, response_token_ids), axis=1)
model_input_gpt.shape


torch.Size([2, 20])

In [77]:
logits, _, v = gpt2_model(model_input_gpt)

In [84]:
logits.shape, logits[:,:-1,:].shape, model_input_gpt[:,1:].shape

(torch.Size([2, 20, 50257]), torch.Size([2, 19, 50257]), torch.Size([2, 19]))

In [85]:
model_input_gpt

tensor(35993, device='cuda:1')

In [ ]:
logprobs_from_logits(logits[:,:-1,:], model_input_gpt[:,1:])[:, -10:]


In [86]:
logprobs_from_logits(logits[:,:-1,:], model_input_gpt[:,1:]).shape

torch.Size([2, 19])

In [87]:
[t for t in reversed(range(10))]

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [51]:
# exports 

class PegasusPPOTrainer:
    """
    The PPO_trainer uses Proximal Policy Optimization to optimise language models.
    """
    
    default_params = {
        "lr": 1.41e-5,
        "adap_kl_ctrl": True, 
        "init_kl_coef":0.2,
        "target": 6,
        "horizon":10000,
        "gamma":1,
        "lam":0.95,
        "cliprange": .2,
        "cliprange_value":.2,
        "vf_coef":.1,
        "batch_size": 256,
        "forward_batch_size": 16,
        "ppo_epochs": 4,    
    } 
    
    def __init__(self, model, ref_model, **ppo_params):
        """
        Initialize PPOTrainer.
        
        Args:
            model (torch.model): Hugging Face transformer GPT2 model with value head
            ref_model (torch.model): Hugging Face transformer GPT2 refrence model used for KL penalty
            ppo_params (dict or None): PPO parameters for training. Can include following keys:
                'lr' (float): Adam learning rate, default: 1.41e-5
                'batch_size' (int): Number of samples per optimisation step, default: 256
                'forward_batch_size' (int): Number of samples forward passed through model at a time, default: 16
                'ppo_epochs' (int): Number of optimisation epochs per batch of samples, default: 4
                'gamma' (float)): Gamma parameter for advantage calculation, default: 1.
                'lam' (float): Lambda parameter for advantage calcualation, default: 0.95
                'cliprange_value' (float): Range for clipping values in loss calculation, default: 0.2
                'cliprange' (float): Range for clipping in PPO policy gradient loss, default: 0.2
                'vf_coef' (float): Scaling factor for value loss, default: 0.1
                'adap_kl_ctrl' (bool): Use adaptive KL control, otherwise linear, default: True
                'init_kl_coef' (float): Initial KL penalty coefficient (used for adaptive and linear control), default: 0.2
                'target' (float): Target KL value for adaptive KL control, default: 6.0
                'horizon' (float): Horizon for adaptive KL control, default: 10000
                
        """
        self.ppo_params = self.default_params
        self.ppo_params.update(ppo_params)
        
        self.ref_model = ref_model
        self.model = model
#         self.optimizer = Adam(model.parameters(), lr=self.ppo_params['lr'])
        self.optimizer = SGD(model.parameters(), lr=self.ppo_params['lr'])
     
        self.kl_ctl = AdaptiveKLController(self.ppo_params['init_kl_coef'],
                                           self.ppo_params['target'],
                                           self.ppo_params['horizon'])


    def step(self, model_input, scores):
        """
        Run a PPO optimisation step.
        
        args:
            query (torch.tensor): tensor containing the encoded queries, shape [batch_size, query_length]
            response (torch.tensor): tensor containing the encoded responses, shape [batch_size, response_length]
            scores (torch.tensor): tensor containing the scores, shape [batch_size]
            
        returns:
            train_stats (dict): a summary of the training statistics
        """

        bs = self.ppo_params['batch_size']
        timing = dict()
        t0 = time.time()
        
        gen_len = model_input["decoder_input_ids"].shape[1] - 1
#         gen_len = response.shape[1]
#         model_input = torch.cat((query, response), axis=1)
        
        t = time.time()
        logprobs, ref_logprobs, values = self.batched_forward_pass(model_input, gen_len)
        timing['time/ppo/forward_pass'] = time.time()-t

        t = time.time()
        rewards, non_score_reward, kl_coef = self.compute_rewards(scores, logprobs, ref_logprobs)
        timing['time/ppo/compute_rewards'] = time.time()-t 
        
        t = time.time() 
        all_stats = []
        idxs = list(range(bs))
        for _ in range(self.ppo_params['ppo_epochs']):
            random.shuffle(idxs)
            for i in range(bs):
                idx = idxs[i]
                train_stats = self.train_minibatch(logprobs[idx:idx+1], 
                                                   values[idx:idx+1],
                                                   rewards[idx:idx+1], 
                                                   split_batch_encoding(model_input, idx, idx+1))
                all_stats.append(train_stats)
        timing['time/ppo/optimize_step'] = time.time()-t
        
        t = time.time()
        train_stats = stack_dicts(all_stats)
        
        # reshape advantages/ratios such that they are not averaged.
        train_stats['policy/advantages'] = torch.flatten(train_stats['policy/advantages']).unsqueeze(0)
        train_stats['policy/ratio'] = torch.flatten(train_stats['policy/ratio']).unsqueeze(0)
        
        stats = self.record_step_stats(scores=scores, logprobs=logprobs, ref_logprobs=ref_logprobs,
                                       non_score_reward=non_score_reward, train_stats=train_stats,
                                       kl_coef=kl_coef)
        stats = stats_to_np(stats)
        timing['time/ppo/calc_stats'] = time.time()-t

        self.kl_ctl.update(stats['objective/kl'], self.ppo_params['batch_size'])

        timing['time/ppo/total'] = time.time()-t0
        stats.update(timing)
        return stats

    def batched_forward_pass(self, model_input, gen_len):
        """Calculate model outputs in multiple batches."""
        bs = self.ppo_params['batch_size']
        fbs = self.ppo_params['forward_batch_size']
        logprobs = []
        ref_logprobs = []
        values = []
        
        for i in range(int(self.ppo_params['batch_size']/fbs)):
#             m_input = model_input[i*fbs:(i+1)*fbs]
            m_input = split_batch_encoding(model_input, i*fbs, (i+1)*fbs)
            logits, _, v = self.model(**m_input)
            ref_logits, _, _ = self.ref_model(**(m_input.to(self.ppo_params['device_ref'])))
            ref_logits = ref_logits.detach().to(self.ppo_params['device_ppo'])
            m_input.to(self.ppo_params['device_ppo'])            
            
            values.append(v[:, -gen_len-1:-1].detach())
            logprobs.append(logprobs_from_logits(logits[:,:-1,:], m_input["decoder_input_ids"][:,1:])[:, -gen_len:].detach())
            ref_logprobs.append(logprobs_from_logits(ref_logits[:,:-1,:], m_input["decoder_input_ids"][:,1:])[:, -gen_len:].detach())
   
        torch.cuda.empty_cache()
        return torch.cat(logprobs), torch.cat(ref_logprobs), torch.cat(values)
    
    def train_minibatch(self, logprobs, values, rewards, model_input):
        """Train one PPO minibatch"""
        loss_p, loss_v, train_stats = self.loss(logprobs, values, rewards, model_input)
        loss = loss_p + loss_v
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return train_stats
    
    def compute_rewards(self, scores, logprobs, ref_logprobs):
        """Compute per token rewards from scores and KL-penalty."""
        kl = logprobs - ref_logprobs
        non_score_reward = -self.kl_ctl.value * kl
        rewards = non_score_reward.clone().detach()
        print("rew: ", rewards.shape, 'score: ', scores.shape)
        rewards[:, -1] += scores
        return rewards, non_score_reward, self.kl_ctl.value

    def loss(self, old_logprobs, values, rewards, model_input):
        """Calculate policy and value losses."""
        lastgaelam = 0
        advantages_reversed = []
        gen_len = model_input["decoder_input_ids"].shape[1] - 1
#         gen_len = response.shape[1]

        for t in reversed(range(gen_len)):
            nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
            delta = rewards[:, t] + self.ppo_params['gamma'] * nextvalues - values[:, t]
            lastgaelam = delta + self.ppo_params['gamma'] * self.ppo_params['lam'] * lastgaelam
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1]).transpose(0, 1)

        returns = advantages + values
        advantages = whiten(advantages)
        advantages = advantages.detach()

        logits, _, vpred = self.model(**model_input)
        logprob = logprobs_from_logits(logits[:,:-1,:], model_input["decoder_input_ids"][:,1:])
#         logprob = logprobs_from_logits(logits[:,:-1,:], model_input[:, 1:])
        
        #only the generation part of the values/logprobs is needed
        logprob, vpred = logprob[:, -gen_len:], vpred[:,-gen_len-1:-1]

        vpredclipped = clip_by_value(vpred,
                                     values - self.ppo_params["cliprange_value"],
                                     values + self.ppo_params["cliprange_value"])

        vf_losses1 = (vpred - returns)**2
        vf_losses2 = (vpredclipped - returns)**2
        vf_loss = .5 * torch.mean(torch.max(vf_losses1, vf_losses2))
        vf_clipfrac =  torch.mean(torch.gt(vf_losses2, vf_losses1).double())

        ratio = torch.exp(logprob - old_logprobs)
        
        pg_losses = -advantages * ratio
        pg_losses2 = -advantages * torch.clamp(ratio,
                                               1.0 - self.ppo_params['cliprange'],
                                               1.0 + self.ppo_params['cliprange'])

        pg_loss = torch.mean(torch.max(pg_losses, pg_losses2))
        pg_clipfrac = torch.mean(torch.gt(pg_losses2, pg_losses).double())
        
        loss = pg_loss + self.ppo_params['vf_coef'] * vf_loss

        entropy = torch.mean(entropy_from_logits(logits))
        approxkl = .5 * torch.mean((logprob - old_logprobs)**2)
        policykl = torch.mean(logprob - old_logprobs)
        return_mean, return_var = torch.mean(returns), torch.var(returns)
        value_mean, value_var = torch.mean(values), torch.var(values)

        stats = dict(
            loss=dict(policy=pg_loss, value=vf_loss, total=loss),
            policy=dict(entropy=entropy, approxkl=approxkl,policykl=policykl, clipfrac=pg_clipfrac,
                        advantages=advantages, advantages_mean=torch.mean(advantages), ratio=ratio),
            returns=dict(mean=return_mean, var=return_var),
            val=dict(vpred=torch.mean(vpred), error=torch.mean((vpred - returns) ** 2),
                     clipfrac=vf_clipfrac, mean=value_mean, var=value_var),
        )
        return pg_loss, self.ppo_params['vf_coef'] * vf_loss, flatten_dict(stats)


    def record_step_stats(self, kl_coef, **data):
        """Record training step statistics."""
        kl = data['logprobs'] - data['ref_logprobs']
        mean_kl = torch.mean(torch.sum(kl, axis=-1))
        mean_entropy = torch.mean(torch.sum(-data['logprobs'], axis=1))
        mean_non_score_reward =torch.mean(torch.sum(data['non_score_reward'], axis=1))
        stats = {
            'objective/kl': mean_kl,
            'objective/kl_dist': kl,
            'objective/logprobs': data['logprobs'],
            'objective/ref_logprobs': data['ref_logprobs'],
            'objective/kl_coef': kl_coef,
            'objective/entropy': mean_entropy,
            'ppo/mean_non_score_reward': mean_non_score_reward,
        }

        for k, v in data['train_stats'].items():
            stats[f'ppo/{k}'] = torch.mean(v, axis=0)
        stats['ppo/val/var_explained'] = 1 - stats['ppo/val/error'] / stats['ppo/returns/var']
        return stats


## Tensor shapes and contents

Debugging tensor shapes and contents usually involves inserting a lot of print statements in the code. To avoid this in the future I add a list of the tensor shapes and contents for reference. If the tensors are sliced or reshaped I list the last shape.

| Name  | Shape   | Content |
|-------|---------|---------|
| `query` | `[batch_size, query_length]`| contains token ids of query|
| `response`| `[batch_size, response_length]`| contains token ids of responses|
| `scores`| `[batch_size]`| rewards of each query/response pair|
| `model_input`| `[batch_size, query_length + response_length]`| combined query and response tokens|
| `m_input`|`[forward_batch_size, query_length + response_length]`| small forward batch of model_input|
| `logits` | `[forward_batch_size, query_length + response_length, vocab_size]`| logits from model outputs|
| `ref_logits`|`[forward_batch_size, query_length + response_length, vocab_size]`| logits from ref_model outputs|
| `logprobs`| `[batch_size, response_length]`| log-probabilities of response tokens |
| `ref_logprobs`| `[batch_size, response_length]`| reference log-probabilities of response tokens |
| `rewards`| `[batch_size, response_length]`| the model rewards incl. kl-score for each token|
| `non_score_reward`| `[batch_size, response_length]`| the model kl-score for each token|

## Model output alignments
Some notes on output alignments, since I spent a considerable time debugging this. All model outputs are shifted by 1 to the model inputs. That means that the logits are shifted by one as well as values. For this reason the logits and values are always shifted one step to the left. This also means we don't have logits for the first input element and so we delete the first input token when calculating the softmax, since we don't have logits predictions. The same applies for the values and we shift them by index one to the left.

## KL-divergence
One question that came up during the implementation was "Why is the KL-divergence just the difference of the log-probs? Where is the probability in front of the log term?". The answer can be found in Sergey Levine's [lecture slides](http://rll.berkeley.edu/deeprlcourse/docs/week_3_lecture_1_dynamics_learning.pdf): To calculate the KL divergence we calculate the expected value of the log term. The probability usually in front of the log-term comes from that expected value and for a set of trajectories we can simply take the mean over the sampled trajectories.